In [4]:
# The code was removed by Watson Studio for sharing.

<div style="text-align: center;margin-bottom:3em">
<h1>Explore and cluster the neighborhoods in Toronto</h1>
</div>

In [1]:
!pip install folium

     |████████████████████████████████| 92kB 6.8MB/s eta 0:00:011


## Imports Libraries

In [2]:
import requests 
import pandas as pd 
import numpy as np 
import random 

from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

import folium 

print('Libraries imported.')

Libraries imported.


## Get Data Frame with Lat & Lng

In [11]:
my_file = project.get_file("mynbtoronto_w_coords.csv")
my_file.seek(0)
df = pd.read_csv(my_file)
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## Define Foursquare Credentials and Version

In [6]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20200314'
LIMIT = 30

## Numbers of neighborhoods

In [7]:
print('The dataframe has {} neighborhoods.'.format(df.shape[0]))

The dataframe has 103 neighborhoods.


## Map of Toronto

In [12]:
addr = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(addr)
lat = location.latitude
lng = location.longitude

my_map = folium.Map(location=[lat, lng], zoom_start=14)


for lat, lng, borough, neighborhood in zip(df['Latitude'],df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False).add_to(my_map)  
    
my_map

## Exploring all neighborhoods

In [15]:
# function for get Venues
def getVenues(names, lat, lng, radius=400):
    
    venues_resp=[]
    for name, lat, lng in zip(names, lat, lng):
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION,lat, lng, radius, LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_resp.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    venues = pd.DataFrame([item for venue_resp in venues_resp for item in venue_resp])
    venues.columns = ['Neighborhood', 'Neighborhood lat', 'Neighborhood lng', 'Venue', 'Venue lat', 'Venue lng', 'Venue category']
    
    return(venues)

venuesOfToronto = getVenues(names=df['Neighborhood'],lat=df['Latitude'],lng=df['Longitude'])
venuesOfToronto.head()

,Neighborhood,Neighborhood lat,Neighborhood lng,Venue,Venue lat,Venue lng,Venue category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


## Shape of Toronto Venues DF

In [16]:
venuesOfToronto.shape

(1045, 7)

## Result group by Neighborhood

In [17]:
venuesOfToronto.groupby('Neighborhood').count()

,Neighborhood lat,Neighborhood lng,Venue,Venue lat,Venue lng,Venue category
Neighborhood,,,,,,
"Adelaide,King,Richmond",30,30,30,30,30,30
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",1,1,1,1,1,1
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",6,6,6,6,6,6
"Alderwood,Long Branch",8,8,8,8,8,8
"Bathurst Manor,Downsview North,Wilson Heights",20,20,20,20,20,20
Bayview Village,2,2,2,2,2,2
"Bedford Park,Lawrence Manor East",23,23,23,23,23,23
Berczy Park,19,19,19,19,19,19


## Cluster Neighborhoods


In [18]:
toronto_top = pd.get_dummies(venuesOfToronto[['Venue category']], prefix="", prefix_sep="")
toronto_top['Neighborhood'] = venuesOfToronto['Neighborhood'] 

fix_columns = [toronto_top.columns[-1]] + list(toronto_top.columns[:-1])

df_torontov = toronto_top.groupby('Neighborhood').mean().reset_index()

#sort the venues in descending order
def popular_venues(row, top_venues):
    row_cat = row.iloc[1:]
    row_cat = row_cat.sort_values(ascending=False)
    
    return row_cat.index.values[0:top_venues]

top_ten_venues = 10

indx = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for i in np.arange(top_ten_venues):
    try:
        columns.append('{}{} Popular venue'.format(i+1, indx[i]))
    except:
        columns.append('{}th Popular venue'.format(i+1))

# create a new dataframe
nbvenues_sorted = pd.DataFrame(columns=columns)
nbvenues_sorted['Neighborhood'] = df_torontov['Neighborhood']

for i in np.arange(df_torontov.shape[0]):
    nbvenues_sorted.iloc[i, 1:] = popular_venues(df_torontov.iloc[i, :], top_ten_venues)

nbvenues_sorted.head()

,Neighborhood,1st Popular venue,2nd Popular venue,3rd Popular venue,4th Popular venue,5th Popular venue,6th Popular venue,7th Popular venue,8th Popular venue,9th Popular venue,10th Popular venue
0,"Adelaide,King,Richmond",Seafood Restaurant,Asian Restaurant,Hotel,Sushi Restaurant,Coffee Shop,Noodle House,Lounge,Pizza Place,Café,Steakhouse
1,Agincourt,Lounge,Latin American Restaurant,Clothing Store,Breakfast Spot,Yoga Studio,Distribution Center,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Coffee Shop,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Pizza Place,Grocery Store,Fried Chicken Joint,Sandwich Place,Liquor Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
4,"Alderwood,Long Branch",Gym,Sandwich Place,Pharmacy,Pizza Place,Pool,Pub,Skating Rink,Coffee Shop,Donut Shop,Dog Run


In [111]:
k = 5

tv_clustering =df_torontov.drop('Neighborhood', 1)
k_means = KMeans(n_clusters=k, random_state=0).fit(tv_clustering)

nbvenues_sorted.insert(0, 'Cluster Labels', k_means.labels_)
df_merge = df

df_merge = df_merge.join(nbvenues_sorted.set_index('Neighborhood'), on='Neighborhood')

#print firts rows
df_merge.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Popular venue,2nd Popular venue,3rd Popular venue,4th Popular venue,5th Popular venue,6th Popular venue,7th Popular venue,8th Popular venue,9th Popular venue,10th Popular venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,3.0,Coffee Shop,Breakfast Spot,Restaurant,Park,Theater,Electronics Store,Spa,Distribution Center,Pub,Bank
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,3.0,Women's Store,Furniture / Home Store,Clothing Store,Boutique,Event Space,Coffee Shop,Shoe Store,Accessories Store,Art Gallery,Arts & Crafts Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,3.0,Coffee Shop,Park,Portuguese Restaurant,Sandwich Place,Distribution Center,Italian Restaurant,Café,Indian Restaurant,Bank,Fried Chicken Joint


In [112]:
df_merge.dropna(inplace=True)
df_merge.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Popular venue,2nd Popular venue,3rd Popular venue,4th Popular venue,5th Popular venue,6th Popular venue,7th Popular venue,8th Popular venue,9th Popular venue,10th Popular venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,3.0,Coffee Shop,Breakfast Spot,Restaurant,Park,Theater,Electronics Store,Spa,Distribution Center,Pub,Bank
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,3.0,Women's Store,Furniture / Home Store,Clothing Store,Boutique,Event Space,Coffee Shop,Shoe Store,Accessories Store,Art Gallery,Arts & Crafts Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,3.0,Coffee Shop,Park,Portuguese Restaurant,Sandwich Place,Distribution Center,Italian Restaurant,Café,Indian Restaurant,Bank,Fried Chicken Joint


## Examining Cluster #1


In [113]:
df_merge.loc[df_merge['Cluster Labels'] == 0, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Popular venue,2nd Popular venue,3rd Popular venue,4th Popular venue,5th Popular venue,6th Popular venue,7th Popular venue,8th Popular venue,9th Popular venue,10th Popular venue
7,North York,0.0,Korean Restaurant,Pool,Baseball Field,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
57,North York,0.0,Baseball Field,Yoga Studio,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
98,Etobicoke,0.0,Pool,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
101,Etobicoke,0.0,Locksmith,Baseball Field,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant


## Examining Cluster #2

In [114]:
df_merge.loc[df_merge['Cluster Labels'] == 1, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Popular venue,2nd Popular venue,3rd Popular venue,4th Popular venue,5th Popular venue,6th Popular venue,7th Popular venue,8th Popular venue,9th Popular venue,10th Popular venue
88,Etobicoke,1.0,Athletics & Sports,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


## Examining Cluster #3

In [115]:
df_merge.loc[df_merge['Cluster Labels'] == 2, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Popular venue,2nd Popular venue,3rd Popular venue,4th Popular venue,5th Popular venue,6th Popular venue,7th Popular venue,8th Popular venue,9th Popular venue,10th Popular venue
6,Scarborough,2.0,Fast Food Restaurant,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
8,East York,2.0,Pizza Place,Fast Food Restaurant,Gastropub,Intersection,Bus Line,Bank,Pharmacy,Pet Store,Dim Sum Restaurant,Electronics Store
10,North York,2.0,Bakery,Pizza Place,Playground,Japanese Restaurant,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
12,Scarborough,2.0,Bar,Yoga Studio,Field,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
14,East York,2.0,Skating Rink,Video Store,Beer Store,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
16,York,2.0,Hockey Arena,Field,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
18,Scarborough,2.0,Mexican Restaurant,Spa,Electronics Store,Rental Car Location,Yoga Studio,Falafel Restaurant,Event Space,Ethiopian Restaurant,Dumpling Restaurant,Donut Shop
23,East York,2.0,Sporting Goods Shop,Bank,Gym,Department Store,Mexican Restaurant,Dessert Shop,Electronics Store,Liquor Store,Restaurant,Burger Joint
26,Scarborough,2.0,Gas Station,Caribbean Restaurant,Bakery,Thai Restaurant,Athletics & Sports,Bank,Hakka Restaurant,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
27,North York,2.0,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Yoga Studio,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant


## Examining Cluster #4

In [116]:
df_merge.loc[df_merge['Cluster Labels'] == 3, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Popular venue,2nd Popular venue,3rd Popular venue,4th Popular venue,5th Popular venue,6th Popular venue,7th Popular venue,8th Popular venue,9th Popular venue,10th Popular venue
1,North York,3.0,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
2,Downtown Toronto,3.0,Coffee Shop,Breakfast Spot,Restaurant,Park,Theater,Electronics Store,Spa,Distribution Center,Pub,Bank
3,North York,3.0,Women's Store,Furniture / Home Store,Clothing Store,Boutique,Event Space,Coffee Shop,Shoe Store,Accessories Store,Art Gallery,Arts & Crafts Store
4,Queen's Park,3.0,Coffee Shop,Park,Portuguese Restaurant,Sandwich Place,Distribution Center,Italian Restaurant,Café,Indian Restaurant,Bank,Fried Chicken Joint
9,Downtown Toronto,3.0,Coffee Shop,Café,Clothing Store,Steakhouse,Sporting Goods Shop,Burger Joint,Ramen Restaurant,Electronics Store,Sandwich Place,College Rec Center
13,North York,3.0,Restaurant,Coffee Shop,Asian Restaurant,Gym,Chinese Restaurant,Japanese Restaurant,Bike Shop,Sporting Goods Shop,Italian Restaurant,Beer Store
15,Downtown Toronto,3.0,Coffee Shop,Italian Restaurant,Gastropub,Restaurant,Hotel,Café,Japanese Restaurant,Performing Arts Venue,Poke Place,Church
17,Etobicoke,3.0,Park,Convenience Store,Coffee Shop,Pizza Place,Cosmetics Shop,Café,Liquor Store,Beer Store,Yoga Studio,Donut Shop
20,Downtown Toronto,3.0,Coffee Shop,Cocktail Bar,Park,French Restaurant,Concert Hall,Department Store,Pub,Restaurant,Lounge,Liquor Store
22,Scarborough,3.0,Coffee Shop,Korean Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant


## Examining Cluster #5

In [117]:
df_merge.loc[df_merge['Cluster Labels'] == 4, df_merge.columns[[1] + list(range(5, df_merge.shape[1]))]]

,Borough,Cluster Labels,1st Popular venue,2nd Popular venue,3rd Popular venue,4th Popular venue,5th Popular venue,6th Popular venue,7th Popular venue,8th Popular venue,9th Popular venue,10th Popular venue
0,North York,4.0,Park,Food & Drink Shop,Fast Food Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
19,East Toronto,4.0,Other Great Outdoors,Trail,Park,Comfort Food Restaurant,Comic Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
21,York,4.0,Park,Women's Store,Market,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
35,East York,4.0,Park,Convenience Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
40,North York,4.0,Park,Grocery Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
46,North York,4.0,Park,Grocery Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop,Dog Run
49,North York,4.0,Park,Bakery,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
64,York,4.0,Park,Convenience Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop
66,North York,4.0,Park,Convenience Store,Bank,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
68,Central Toronto,4.0,Park,Locksmith,Comfort Food Restaurant,Comic Shop,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant


## Map of clusters

In [118]:
mapOfClusters = folium.Map(location=[lat, lng], zoom_start=12)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
colors_matrix = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, nbg, cluster in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Neighborhood'], df_merge['Cluster Labels']):
    
    if(cluster == 4.0): cluster = 4
    elif(cluster == 3.0): cluster = 3
    elif (cluster == 2.0): cluster = 2
    elif (cluster == 1.0): cluster = 1
    else: cluster = 0
        
    label = folium.Popup(str(nbg) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors_matrix[cluster],
        fill=True,
        fill_color=colors_matrix[cluster],
        fill_opacity=0.5).add_to(mapOfClusters)

mapOfClusters